# Architekturkonzepte: Quick Start Guide

Dieses Notebook bietet einen schnellen Einstieg in verschiedene Architekturkonzepte für LLM-Anwendungen, die im Workshop vorgestellt wurden. Verwenden Sie diese Beispiele, um die Konzepte schnell in eigenen Projekten anzuwenden.

## Inhalt
1. Einfacher Agent
2. ReAct-Pattern
3. Graph-RAG
4. Hierarchical RAG

In [ ]:
# Benötigte Bibliotheken installieren
!pip install langchain langchain-community langchain-openai langgraph python-dotenv -q

# Umgebungsvariablen laden
import os

from dotenv import load_dotenv

load_dotenv()

# API-Schlüssel überprüfen
if os.getenv("OPENAI_API_KEY"):
    print("✓ OpenAI API-Schlüssel konfiguriert")
else:
    print("⚠ OpenAI API-Schlüssel fehlt! Bitte in .env konfigurieren.")

## 1. Einfacher Agent

Dieses Beispiel zeigt, wie Sie einen einfachen Agenten erstellen, der Tools verwenden kann.

In [ ]:
from langchain_openai import OpenAI
from langchain.tools import tool
from langchain.agents import AgentType, initialize_agent, load_tools

# LLM initialisieren
llm = OpenAI(temperature=0)

# Tools laden
tools = load_tools(["llm-math"], llm=llm)


# Eigenes Tool erstellen
@tool
def wetter_info(ort: str) -> str:
    """Gibt Wetterinformationen für einen bestimmten Ort zurück. Der Ort sollte als Eingabe übergeben werden."""
    # In einem realen Szenario würden Sie hier eine Wetter-API abfragen
    # Dies ist nur ein Beispiel
    return f"In {ort} sind es heute 22°C und sonnig."


# Tool zur Liste hinzufügen
tools.append(wetter_info)

# Agent initialisieren
agent = initialize_agent(
    tools,
    llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

# Agent testen
agent.invoke("Wie ist das Wetter in Berlin und berechne 25 hoch 0.5")

## 2. ReAct Pattern

Das ReAct-Muster (Reasoning and Acting) ermöglicht dem LLM, sein Denken explizit zu verbalisieren und rational Entscheidungen zu treffen.

In [ ]:
# ReAct-Agent für komplexe Aufgaben
from langchain.agents import AgentType, initialize_agent, load_tools
from langchain_openai import OpenAI

# LLM initialisieren
llm = OpenAI(temperature=0)

# Tools laden
# Wir verwenden hier serpapi, falls konfiguriert; ansonsten DuckDuckGo
search_tools = []
if os.getenv("SERPAPI_API_KEY"):
    search_tools = load_tools(["serpapi"])
else:
    search_tools = load_tools(["ddg-search"])

# Mathematik-Tool hinzufügen
tools = search_tools + load_tools(["llm-math"], llm=llm)

# ReAct-Agent initialisieren
react_agent = initialize_agent(
    tools,
    llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

# Agent mit einer komplexen Anfrage testen, die mehrere Schritte erfordert
react_agent.invoke("Wer ist der aktuelle Bundeskanzler und in welchem Jahr wurde er geboren? Addiere dann 25 zu diesem Jahr.")

## 3. Graph-RAG

Graph-RAG kombiniert Graphstrukturen mit Retrieval Augmented Generation, um Beziehungen zwischen Dokumenten zu modellieren.

In [ ]:
from typing import TypedDict, List
from langgraph.graph import StateGraph
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_community.vectorstores import Chroma


# Zustandsdefinition für Graph-RAG
class GraphState(TypedDict):
    query: str
    context: List[str]
    answer: str


# LLM und Embeddings initialisieren
llm = ChatOpenAI(temperature=0)
embeddings = OpenAIEmbeddings()

# Beispieldaten für die Vektordatenbank
sample_texts = [
    "Berlin ist die Hauptstadt von Deutschland und hat etwa 3,7 Millionen Einwohner.",
    "München ist die Hauptstadt von Bayern und bekannt für das Oktoberfest.",
    "Hamburg ist die zweitgrößte Stadt Deutschlands und ein wichtiger Hafen.",
    "Frankfurt am Main ist ein bedeutendes Finanzzentrum in Europa.",
    "Köln ist bekannt für seinen gotischen Dom und den Karneval."
]

# Vektordatenbank erstellen
vectorstore = Chroma.from_texts(sample_texts, embeddings)


# Knoten-Funktionen definieren
def retrieve(state: GraphState) -> GraphState:
    """Dokumente aus der Vektordatenbank abrufen"""
    query = state["query"]
    docs = vectorstore.similarity_search(query, k=2)
    return {"context": [doc.page_content for doc in docs]}


def generate_answer(state: GraphState) -> GraphState:
    """Antwort basierend auf dem Kontext generieren"""
    query = state["query"]
    context = state["context"]

    prompt = ChatPromptTemplate.from_template(
        """Du bist ein hilfreicher Assistent. 
        Verwende den folgenden Kontext, um die Frage zu beantworten.
        
        Kontext: {context}
        
        Frage: {query}
        """
    )

    chain = prompt | llm | StrOutputParser()
    answer = chain.invoke({"context": "\n".join(context), "query": query})

    return {"answer": answer}


# Graph erstellen
graph = StateGraph(GraphState)

# Knoten hinzufügen
graph.add_node("retrieve", retrieve)
graph.add_node("generate", generate_answer)

# Kanten definieren
graph.set_entry_point("retrieve")
graph.add_edge("retrieve", "generate")
graph.set_finish_point("generate")

# Graph kompilieren und ausführen
chain = graph.compile()
result = chain.invoke({"query": "Was ist die Hauptstadt von Deutschland?", "context": [], "answer": ""})
print("\nAntwort:", result["answer"])

## 4. Hierarchical RAG

Hierarchical RAG organisiert Informationen in Hierarchien, um den Suchraum effizient einzugrenzen.

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

# LLM initialisieren
llm = ChatOpenAI(temperature=0)

# Hierarchische Dokument-Struktur (vereinfacht)
document_hierarchy = {
    "level1": [
        {"id": "doc1", "summary": "Deutschlands Großstädte und ihre Bedeutung"},
        {"id": "doc2", "summary": "Europäische Hauptstädte im Vergleich"}
    ],
    "level2": {
        "doc1": [
            {"id": "doc1_section1", "title": "Berlin als Hauptstadt"},
            {"id": "doc1_section2", "title": "Hamburg als Handelsmetropole"}
        ],
        "doc2": [
            {"id": "doc2_section1", "title": "Berlin im europäischen Kontext"},
            {"id": "doc2_section2", "title": "Paris als Kulturzentrum"}
        ]
    },
    "level3": {
        "doc1_section1": "Berlin ist die Hauptstadt und bevölkerungsreichste Stadt Deutschlands. Mit rund 3,7 Millionen Einwohnern ist Berlin auch die größte Stadt der Europäischen Union.",
        "doc1_section2": "Hamburg ist mit 1,8 Millionen Einwohnern die zweitgrößte Stadt Deutschlands und ein bedeutendes Wirtschafts- und Handelszentrum in Nordeuropa.",
        "doc2_section1": "Im Vergleich zu anderen europäischen Hauptstädten hat Berlin eine besondere Geschichte aufgrund der deutschen Teilung im 20. Jahrhundert.",
        "doc2_section2": "Paris, die Hauptstadt Frankreichs, gilt als eines der wichtigsten Kulturzentren Europas mit berühmten Museen wie dem Louvre und dem Centre Pompidou."
    }
}


def hierarchical_search(query: str) -> str:
    """Führt eine hierarchische Suche durch"""

    # Level 1: Auswahl des relevanten Dokuments
    level1_prompt = ChatPromptTemplate.from_template(
        """Gegeben sind die folgenden Dokumentzusammenfassungen:
        {summaries}
        
        Für die Anfrage: {query}
        Gib die ID des relevantesten Dokuments zurück. Antworte nur mit der ID."""
    )

    summaries = "\n".join([f"ID: {doc['id']}, Zusammenfassung: {doc['summary']}"
                           for doc in document_hierarchy["level1"]])

    level1_chain = level1_prompt | llm | StrOutputParser()
    selected_doc = level1_chain.invoke({"summaries": summaries, "query": query})
    print(f"Ausgewähltes Dokument (Level 1): {selected_doc}")

    # Level 2: Auswahl des relevanten Abschnitts
    level2_prompt = ChatPromptTemplate.from_template(
        """Gegeben sind die folgenden Abschnitte aus dem Dokument {doc_id}:
        {sections}
        
        Für die Anfrage: {query}
        Gib die ID des relevantesten Abschnitts zurück. Antworte nur mit der ID."""
    )

    sections = "\n".join([f"ID: {section['id']}, Titel: {section['title']}"
                          for section in document_hierarchy["level2"][selected_doc]])

    level2_chain = level2_prompt | llm | StrOutputParser()
    selected_section = level2_chain.invoke({"doc_id": selected_doc, "sections": sections, "query": query})
    print(f"Ausgewählter Abschnitt (Level 2): {selected_section}")

    # Level 3: Abrufen des detaillierten Inhalts
    content = document_hierarchy["level3"][selected_section]

    # Generieren der finalen Antwort
    answer_prompt = ChatPromptTemplate.from_template(
        """Basierend auf dem folgenden Text, beantworte die Frage.
        
        Text: {content}
        
        Frage: {query}
        
        Antwort:"""
    )

    answer_chain = answer_prompt | llm | StrOutputParser()
    answer = answer_chain.invoke({"content": content, "query": query})

    return answer


# Hierarchische Suche testen
result = hierarchical_search("Was ist besonders an Berlin?")
print("\nFinale Antwort:")
print(result)

## 5. Bonus: Kombinierte Architektur (ReAct + Graph-RAG)

Dieses Beispiel zeigt, wie Sie ReAct und Graph-RAG kombinieren können, um einen leistungsfähigen Agenten zu erstellen.

In [ ]:
from langchain.tools import tool
from langchain.agents import AgentType, initialize_agent
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_community.vectorstores import Chroma

# LLM und Embeddings initialisieren
llm = ChatOpenAI(temperature=0)
embeddings = OpenAIEmbeddings()

# Beispieldaten für die Vektordatenbank
sample_texts = [
    "Berlin ist die Hauptstadt von Deutschland und hat etwa 3,7 Millionen Einwohner.",
    "München ist die Hauptstadt von Bayern und bekannt für das Oktoberfest.",
    "Hamburg ist mit 1,8 Millionen Einwohnern die zweitgrößte Stadt Deutschlands.",
    "Frankfurt am Main ist ein bedeutendes Finanzzentrum in Europa.",
    "Köln ist bekannt für seinen gotischen Dom und den Karneval."
]

# Vektordatenbank erstellen
vectorstore = Chroma.from_texts(sample_texts, embeddings)


# Tool für Dokumentensuche erstellen
@tool
def suche_dokument(query: str) -> str:
    """Sucht nach relevanten Dokumenten basierend auf einer Anfrage."""
    docs = vectorstore.similarity_search(query, k=2)
    return "\n\n".join([doc.page_content for doc in docs])


# Tool für mathematische Berechnungen
math_tools = load_tools(["llm-math"], llm=llm)

# Alle Tools kombinieren
tools = [suche_dokument] + math_tools

# ReAct-Agent erstellen, der die Tools verwendet
combined_agent = initialize_agent(
    tools,
    llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

# Agent testen mit einer Anfrage, die sowohl Dokumentensuche als auch Mathematik erfordert
combined_agent.invoke(
    "Welche Stadt ist die bevölkerungsreichste in Deutschland und wie viel ist ihre Einwohnerzahl in Millionen zum Quadrat?"
)

## Zusammenfassung

In diesem Quick Start Guide haben Sie kennengelernt:
- Wie Sie einfache Agenten mit Tools erstellen
- Wie das ReAct-Pattern funktioniert und implementiert wird
- Wie Graph-RAG mit LangGraph umgesetzt wird
- Wie hierarchische Suche in strukturierten Dokumenten funktioniert
- Wie Sie verschiedene Architekturansätze kombinieren können

Diese Beispiele dienen als Ausgangspunkt für Ihre eigenen LLM-Anwendungen. Experimentieren Sie mit verschiedenen Kombinationen und passen Sie die Beispiele an Ihre speziellen Anforderungen an.